In [1]:
import nltk

import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from transformers import BertTokenizerFast, BertModel, AdamW
from transformers import AutoTokenizer, AutoModel, AdamW
from torch.utils.data import TensorDataset, DataLoader
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import time
from tqdm.notebook import tqdm



device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

eng_dict = nltk.corpus.words.words('en')

cuda:0


In [2]:


tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/transformers/bert-base-uncased",
                                          do_lower_case=True)
# tokenizer.save_pretrained('/kaggle/input/transformers/bert-base-uncased')

bert_model = AutoModel.from_pretrained('/kaggle/input/transformers/bert-base-uncased')
# bert_model.save_pretrained('bert-base-uncased')

            


# initialguess for the archetecture is [823, 512, 256, 128, 1]
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, output_dim):
        super(MLP, self).__init__()
        self.fc1  = nn.Linear(input_dim, hidden_dim1)
        self.fc2  = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3  = nn.Linear(hidden_dim2, hidden_dim3)
        self.fc4  = nn.Linear(hidden_dim3, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.batchnorm1 = nn.BatchNorm1d(hidden_dim1)
        self.batchnorm2 = nn.BatchNorm1d(hidden_dim2)
        self.batchnorm3 = nn.BatchNorm1d(hidden_dim3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.fc4(x)
        return x
            
        
class CombinedModel(nn.Module):
    def __init__(self, bert_model, mlp):
        super(CombinedModel, self).__init__()
        self.bert = bert_model
        self.mlp = mlp

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state[:, 0, :]
        logits = self.mlp(last_hidden_state)
        return logits
            
    

def validation(model, val_loader, loss_fn):
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    y_pred = []
    y_true = []
    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids, attention_mask, labels = [t.to(device) for t in batch]
            outputs = model(input_ids, attention_mask=attention_mask) #, labels=labels)
            loss = loss_fn(outputs.to(device), labels.type(torch.LongTensor).to(device))
            val_loss += loss
            val_correct += (torch.max(outputs.data, dim=1)[1] == labels).sum().item()
            val_total += len(labels)
            
            y_pred.extend(torch.softmax(outputs.data, dim=1)[:, 1].cpu().numpy())  # Assuming class 1 probability
            y_true.extend(labels.cpu().numpy())

            val_correct += (torch.max(outputs.data, dim=1)[1] == labels).sum().item()
            val_total += len(labels)
    
    y_pred_label = [int(num >= 0.5) for num in y_pred]
    roc_auc = roc_auc_score(y_true, y_pred_label)
    precision = precision_score(y_true, y_pred_label)
    recall = recall_score(y_true, y_pred_label)
    f1 = f1_score(y_true, y_pred_label)
    val_loss = val_loss / val_total
    val_accuracy = 100 * val_correct / val_total
    return y_pred, y_pred_label, val_loss, val_accuracy, roc_auc, precision, recall, f1


def testing(model, test_loader):
    model.eval()
    y_pred = []
    with torch.no_grad():
        for batch in tqdm(test_loader):
            input_ids, attention_mask = [t.to(device) for t in batch]
            outputs = model(input_ids, attention_mask=attention_mask) 
            y_pred.extend(torch.softmax(outputs.data, dim=1)[:, 1].cpu().numpy())  # Assuming class 1 probability   
    y_pred_label = [int(num >= 0.5) for num in y_pred]
    return y_pred, y_pred_label


def tokenize_and_encode(tokenizer, comments, max_length=128): 
    input_ids = [] 
    attention_masks = [] 

    for comment in comments: 
        encoded_dict = tokenizer.encode_plus( 
            comment,  
            add_special_tokens=True, 
            max_length=max_length, 
            pad_to_max_length=True, 
            return_attention_mask=True, 
            return_tensors='pt'
        ) 

        input_ids.append(encoded_dict['input_ids']) 
        attention_masks.append(encoded_dict['attention_mask']) 

    input_ids = torch.cat(input_ids, dim=0) 
    attention_masks = torch.cat(attention_masks, dim=0) 

    return input_ids, attention_masks


/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


# My data model metrics

In [4]:

tokenizer = AutoTokenizer.from_pretrained('../input/transformers/bert-base-uncased', do_lower_case=True)

bert_model = AutoModel.from_pretrained('../input/transformers/bert-base-uncased')
num_layers = len(bert_model.encoder.layer)  # Adjust based on your BERT model

mlp = MLP(768, 512, 256, 128, 2)
model = CombinedModel(bert_model, mlp).to(device)
checkpoint = torch.load('/kaggle/input/llm-detect-human-ai-2-1/checkpoint.pth')
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()




CombinedModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

### Examples of AI generated texts


In [5]:
data = pd.DataFrame(columns = ['text', 'generated'])

data['generated'] = [1 for i in range(10)] + [0 for i in range(10)] + [-1 for i in range(10)] 


In [6]:
# Gemini
string_gemini = [
                    'The rapid advancement of artificial intelligence (AI) has blurred the lines between human and machine-generated content. While AI has revolutionized various fields, it has also posed a significant challenge: distinguishing human-generated text from AI-generated content. This difficulty stems from several interconnected factors, including the sophistication of AI language models, the complexity of human language, and the subjective nature of evaluation.One of the primary reasons for the difficulty in detecting AI-generated text is the increasing sophistication of AI language models. These models, such as GPT-3 and LaMDA, are trained on massive datasets of human-generated text, enabling them to learn patterns, styles, and nuances of human language. As a result, they can produce text that is remarkably similar to human-written content, making it challenging to identify their artificial origin. These models can even generate creative and coherent text, often surpassing human capabilities in certain tasks.Moreover, the complexity of human language itself contributes to the difficulty of detection. Human language is filled with ambiguities, nuances, and context-dependent meanings. It is often characterized by figurative language, idioms, and cultural references that can be challenging for AI models to fully comprehend and replicate. While AI models have made significant strides in understanding and generating human-like text, they may still struggle to capture the subtleties and idiosyncrasies of human communication.Furthermore, the subjective nature of evaluation poses another hurdle in distinguishing human-generated text from AI-generated content. The assessment of text quality is often influenced by personal biases, cultural preferences, and individual interpretations. What one person may consider high-quality writing may be perceived differently by another. As a result, it can be challenging to establish objective criteria for distinguishing between human and AI-generated text.In conclusion, the difficulty in detecting AI-generated text stems from the sophistication of AI language models, the complexity of human language, and the subjective nature of evaluation. While AI has made remarkable progress in generating human-like text, it is still not perfect. As AI continues to evolve, it is likely that the challenge of distinguishing human-generated text from AI-generated content will become even more complex.',
                    'Consciousness, that elusive, subjective realm of experience, has captivated philosophers, scientists, and mystics for millennia. It remains one of the greatest unsolved mysteries of the universe. How does a collection of neurons, firing electrical signals, give rise to the rich tapestry of thoughts, feelings, and sensations that we call consciousness?The scientific understanding of consciousness is still in its infancy. Neurobiological studies have identified brain regions involved in various aspects of conscious experience, such as attention, memory, and self-awareness. However, the precise mechanisms underlying these processes remain elusive. Some theories propose that consciousness arises from complex patterns of neural activity, while others suggest that it is a fundamental property of the universe.Beyond the scientific realm, consciousness has also been explored in philosophical and spiritual traditions. Many philosophers have argued that consciousness is a unique aspect of human existence, distinct from the material world. Some religious and mystical traditions believe that consciousness is connected to a higher spiritual realm, transcending the physical limitations of the body.The question of consciousness raises profound philosophical questions about the nature of reality, the self, and the meaning of life. Is consciousness a product of the brain, or is it something more? Is it possible for machines to become conscious? What is the relationship between consciousness and free will?As we continue to explore the mysteries of consciousness, we may find that it is not a single phenomenon but a complex interplay of biological, psychological, and perhaps even metaphysical factors. The quest to understand consciousness is a journey that takes us to the very heart of what it means to be human.',
                    'Particle physics, the study of the fundamental constituents of matter and their interactions, has witnessed remarkable advancements in recent years. The Large Hadron Collider (LHC), the most powerful particle accelerator ever built, has played a pivotal role in these breakthroughs. One of the most significant discoveries of the LHC was the confirmation of the existence of the Higgs boson, a particle predicted by the Standard Model of particle physics. The Higgs boson is crucial for understanding how other particles acquire mass. Its discovery was a major triumph for the Standard Model, providing strong evidence for its validity. Beyond the Higgs boson, the LHC has also explored other aspects of particle physics. Scientists have been searching for evidence of new particles and forces that could extend the Standard Model and explain phenomena like dark matter and dark energy. While no definitive discoveries have been made yet in this area, the LHC continues to probe the frontiers of our understanding. Looking ahead, the future of particle physics is filled with exciting possibilities. The LHC is expected to continue its operations for many years to come, with upgrades planned to increase its energy and luminosity. These upgrades could lead to new discoveries and a deeper understanding of the fundamental forces of nature. In addition to the LHC, other particle physics experiments are also making significant contributions. Neutrino experiments are exploring the properties of neutrinos, elusive particles that can pass through matter almost undetected. Dark matter experiments are searching for evidence of dark matter particles, which make up the majority of matter in the universe but have yet to be directly observed. As particle physics continues to evolve, we can expect to see further advancements in our understanding of the fundamental building blocks of the universe. The discoveries made in this field have the potential to revolutionize our knowledge of the cosmos and shape the future of technology.'
                ]

# ChatGPT
string_chatgpt = [
                    'Education systems in different countries reflect their unique societal values, priorities, and historical contexts. The systems in the USA, Finland, and India, for example, offer vastly different structures and approaches, each with its strengths and challenges. While it is tempting to rank these systems and declare one as the "best," such comparisons often overlook important cultural and contextual factors. The United States: Diversity and Flexibility. The U.S. education system is characterized by its diversity, flexibility, and emphasis on individualism. It allows students and parents to choose between public, private, charter, and homeschooling options, creating a broad spectrum of learning environments. The U.S. excels in higher education, with many of its universities ranked among the best globally, fostering innovation and research. However, at the K-12 level, disparities in funding, access to resources, and quality vary widely, especially between wealthy and low-income districts. Standardized testing is a major aspect of assessment, but this has been criticized for narrowing curricula and placing too much focus on test performance rather than holistic learning. Finland: Equity and Well-Being. Finland’s education system is renowned for its emphasis on equity, student well-being, and trust in teachers. Finnish schools provide a relaxed learning environment with less focus on standardized testing and more on creativity, collaboration, and critical thinking. Teachers, who are highly respected professionals, have a great deal of autonomy in designing their lessons. Finnish students start formal schooling at a later age, and the system focuses on reducing stress and promoting lifelong learning. While Finland consistently ranks high in global education assessments like PISA (Programme for International Student Assessment), the system has come under criticism for being slow to adapt to the growing needs of a globalized and increasingly digital world. India: Scale and Aspirations. India faces unique challenges and opportunities in its education system due to its vast population and socio-economic diversity. The country has made significant strides in improving access to education through policies like the Right to Education Act, which mandates free and compulsory education for children aged 6 to 14. India’s system places a strong emphasis on rote learning and high-stakes exams, especially in secondary education. However, significant disparities exist between urban and rural areas, and access to quality education is often limited in underdeveloped regions. Despite these challenges, India produces a large number of STEM graduates, contributing to global tech industries and innovation. Why Declaring One System the Best is Misguided. Declaring any of these systems the "best" overlooks the fact that each country’s education model is shaped by unique cultural, economic, and social contexts. The U.S. values choice and flexibility, Finland prioritizes equity and teacher autonomy, and India must address challenges tied to its vast population and resource disparities. Each system has strengths that align with the needs of its society and challenges that are deeply rooted in historical and socio-economic factors. Moreover, what works in one country may not be effective or relevant in another due to differences in culture, government structure, and societal goals. In conclusion, education is not one-size-fits-all. Instead of ranking systems, it’s more productive to analyze each within its own context and explore ways to learn from their successes and failures.',
                    'Over the last 500 years, art has undergone a remarkable transformation, shaped by evolving societal values, technological advancements, and shifting philosophical outlooks. The Renaissance of the 16th century marked a pivotal moment, where artists like Leonardo da Vinci and Michelangelo prioritized ideal beauty, humanism, and scientific precision. As art moved into the Baroque and Romantic periods, it reflected grandeur, emotion, and power, with artists like Caravaggio and Goya capturing intense human experiences. By the late 19th century, the advent of Impressionism and later Modernism broke away from traditional forms of realism, with figures like Monet and Picasso redefining art’s boundaries through abstraction, experimentation, and a focus on personal expression. The 20th century ushered in a pluralistic approach, embracing diverse styles such as Surrealism, Abstract Expressionism, and Minimalism, where artists like Dalí and Pollock further pushed the limits of form and concept. Contemporary art, influenced by globalization and digital technology, now explores a vast array of mediums, from digital installations to performance art, challenging conventional notions of creativity and authorship. Today, art is more democratic and accessible than ever, with social media and global platforms allowing artists to reach diverse audiences. What was once confined to elite patrons and institutions has become a form of mass cultural expression, where the line between fine art and popular culture continues to blur. Art is no longer bound by rigid definitions; instead, it thrives as an ever-evolving reflection of society’s complexities, ideas, and technological advances, making it impossible to declare a singular approach or movement as definitive.',
                    'The geopolitical climate of the 21st century is defined by a complex interplay of global powers, economic interdependence, technological advancements, and rising nationalism. Unlike the bipolar world of the Cold War, today\'s global landscape is multipolar, with the United States, China, and Russia playing dominant but distinct roles. The U.S. remains a leading military and economic force, but its influence is increasingly challenged by China\'s rapid rise as a global economic powerhouse, technological leader, and regional military assertor in Asia, especially in the South China Sea. Russia, while economically weaker, wields significant influence through energy exports, military interventions, and cyber capabilities, notably seen in its involvement in Ukraine and the broader post-Soviet space. At the same time, the European Union, despite internal challenges such as Brexit and differing national interests, continues to be a major economic bloc and diplomatic force, especially in addressing climate change and human rights issues. The 21st century also witnesses the growing importance of emerging markets, particularly India, Brazil, and South Africa, as key players in global trade and diplomacy. However, the rise of populist and nationalist movements in many parts of the world, including the U.S., Europe, and Latin America, reflects a backlash against globalization, leading to increasing protectionism, anti-immigration policies, and weakening multilateral institutions like the UN and WTO. Climate change has further added to geopolitical tensions, exacerbating resource conflicts, migration, and environmental disasters. Meanwhile, technological advancements, particularly in artificial intelligence, cyber warfare, and space exploration, have introduced new dimensions to global power struggles. Cybersecurity, information warfare, and control over digital infrastructure have become central to national security, with major powers vying for dominance in the tech landscape. The geopolitical environment is thus characterized by shifting alliances, regional conflicts, the rise of authoritarianism, and a global power struggle that increasingly takes place in both physical and digital realms. The 21st century presents a world where cooperation and competition coexist, and global stability is fragile, requiring new forms of diplomacy and governance to address the unprecedented challenges of an interconnected world.',
                    'Spending all one\'s time on computers gaming presents a mix of both pros and cons, with impacts on mental, social, and physical well-being. On the positive side, gaming can enhance cognitive skills such as problem-solving, strategic thinking, and hand-eye coordination, especially in fast-paced or complex games that require multitasking. It also provides a sense of achievement, stress relief, and social interaction, particularly in multiplayer games where players can collaborate with or compete against others globally, fostering teamwork and community. Some even turn gaming into a career through professional eSports or content creation. However, there are significant downsides to excessive gaming. Physically, long hours in front of a screen can lead to sedentary behavior, increasing the risk of obesity, poor posture, eye strain, and repetitive strain injuries. Mentally, gaming can contribute to addiction, leading to neglect of real-world responsibilities such as school, work, or social relationships. Over-reliance on gaming for social interaction can diminish face-to-face communication skills and reduce meaningful social engagement. Additionally, excessive gaming can contribute to mental health issues like anxiety, depression, and stress, especially when used as an escape mechanism. Thus, while gaming offers entertainment and cognitive benefits, it needs to be balanced with physical activity, social interaction, and other responsibilities to prevent its negative impacts on health and well-being.'
                ]

# claude

string_claude = [
                    'The landscape of AI chatbots has been rapidly evolving, with Gemini, Claude, and ChatGPT emerging as prominent contenders, each offering unique strengths and capabilities. ChatGPT, developed by OpenAI, initially set the standard for large language models with its impressive natural language understanding and generation abilities, gaining widespread popularity and showcasing the potential of AI in various applications. Claude, created by Anthropic, entered the scene with a focus on safety and ethical considerations, demonstrating advanced reasoning skills and a nuanced approach to complex topics. Gemini, Google\'s latest AI model, aims to push the boundaries further by integrating multimodal capabilities, allowing it to process and generate both text and visual content seamlessly. While all three chatbots excel in general conversation and task completion, they have distinct characteristics that set them apart. ChatGPT is known for its versatility and extensive knowledge base, making it adept at a wide range of tasks from creative writing to coding assistance. Claude stands out for its strong emphasis on safety and alignment with human values, often providing more cautious and ethically-aware responses. Gemini\'s multimodal approach gives it an edge in tasks that involve both textual and visual elements, potentially opening up new possibilities for AI applications. In terms of performance, recent comparisons suggest that these models are fairly competitive, with each excelling in different areas. ChatGPT\'s frequent updates and extensive training data allow it to maintain relevance across diverse topics. Claude\'s thoughtful and nuanced responses often shine in complex reasoning tasks and discussions on sensitive subjects. Gemini\'s multimodal capabilities offer unique advantages in scenarios requiring visual understanding alongside text generation. However, it\'s important to note that the AI landscape is rapidly changing, with frequent updates and improvements to these models. Ultimately, the choice between Gemini, Claude, and ChatGPT may depend on specific use cases, ethical considerations, and the desired balance between capabilities and safety features. As these AI chatbots continue to evolve, they collectively represent the cutting edge of natural language processing technology, each contributing to the advancement of human-AI interaction in its own unique way.',
                    'The remotest inhabited place on Earth with a population exceeding 1,000 people is widely considered to be Tristan da Cunha, a volcanic island group situated in the South Atlantic Ocean. This British Overseas Territory lies approximately 1,500 miles (2,400 kilometers) from the nearest continental land mass in South Africa and 2,088 miles (3,360 kilometers) from South America, earning it the title of the most isolated inhabited archipelago in the world. The main island, also named Tristan da Cunha, is home to the entire population of around 250 people, all of whom reside in the settlement of Edinburgh of the Seven Seas. Despite not meeting the 1,000-person threshold specified, it\'s worth noting as the largest settlement that truly embodies extreme remoteness. For a location meeting both the remoteness and population criteria, we might consider Longyearbyen on the Norwegian archipelago of Svalbard, with approximately 2,400 residents. Located at 78 degrees north latitude, it\'s the world\'s northernmost settlement with more than 1,000 inhabitants, situated about 650 miles (1,050 kilometers) from the North Pole. Longyearbyen faces extreme Arctic conditions, including polar nights that last for four months and midnight sun for an equal duration. The community, originally established as a coal mining town, has evolved into a hub for Arctic research, tourism, and satellite technology. Life in Longyearbyen is marked by unique challenges, including the constant threat of polar bears, which necessitates carrying a rifle when venturing outside the settlement limits. The permafrost environment also presents difficulties for infrastructure and daily living. Despite these challenges, Longyearbyen maintains a vibrant community with modern amenities, including a university center, hospital, and airport, which serves as a lifeline to the outside world. The extreme isolation of places like Tristan da Cunha and Longyearbyen underscores human adaptability and resilience. These communities have developed distinct cultures and ways of life, shaped by their remote locations and harsh environments. They serve as testament to humanity\'s enduring spirit of exploration and the capacity to thrive in even the most isolated corners of our planet, offering unique perspectives on sustainability, community, and the relationship between humans and their environment.',
                    'Machine learning (ML) has emerged as a powerful tool in the field of quantum many-body physics, offering new approaches to tackle complex problems that have long challenged traditional computational methods. The application of ML techniques, such as neural networks and deep learning algorithms, to quantum systems has opened up exciting possibilities for understanding and simulating intricate quantum phenomena that involve multiple interacting particles. These ML approaches have shown remarkable success in areas such as quantum state tomography, discovering new quantum phases of matter, and optimizing quantum control protocols. By leveraging the ability of ML models to recognize patterns and extract relevant features from vast amounts of data, researchers can now explore quantum systems with unprecedented efficiency and accuracy. This synergy between ML and quantum physics has led to significant advancements, including improved predictions of material properties, more efficient quantum simulations, and novel insights into quantum phase transitions. However, the integration of ML into quantum many-body physics is not without controversy. On one hand, proponents argue that ML techniques offer a much-needed computational boost to a field often limited by the exponential complexity of quantum systems, potentially leading to breakthroughs in our understanding of fundamental physics and the development of new quantum technologies. ML methods can handle high-dimensional data more effectively than traditional approaches, making them particularly suited for the study of entangled quantum states. On the other hand, critics raise concerns about the interpretability of ML models in physics, questioning whether the black-box nature of some ML algorithms might obscure the underlying physical principles at play. There\'s also a debate about the reliability of ML predictions in quantum systems, especially when extrapolating beyond the training data. Furthermore, some physicists worry that an over-reliance on ML might lead to a reduction in theoretical understanding, as researchers may become too dependent on data-driven approaches at the expense of developing fundamental physical insights. Despite these concerns, the consensus among many researchers is that the benefits of ML in quantum many-body physics outweigh the potential drawbacks. The key lies in using ML as a complementary tool to enhance, rather than replace, traditional physical methods and theoretical understanding. By combining the strengths of ML with rigorous physical principles and experimental validation, scientists can harness the power of data-driven approaches while maintaining the interpretability and physical intuition that are crucial to scientific progress. As the field continues to evolve, it will be essential to develop ML techniques that are more transparent and physically motivated, ensuring that the insights gained through these methods contribute meaningfully to our understanding of quantum many-body systems. Ultimately, the judicious application of ML in quantum physics represents a promising frontier that, when navigated carefully, has the potential to revolutionize our ability to explore and harness the quantum world.'
                ]


In [7]:
# Human written
string_human = [
                    'Any culture tells you how to live your one and only life: to wit, as everyone else does. Probably most cultures prize, as ours rightly does, making a contribution by working hard at work that you love; being in the know, and intelligent; gathering a surplus; and loving your family above all, and your dog, your cat; bird-watching. Beyond those things our culture may place a special focus on money, on celebrity, on physical beauty. These are not universal. Elsewhere it might be: You wear the best shoes you can afford, you seek to know Rome’s best restaurants and their staffs, drive the best car, and vacation on Tenerife. And what a cook you are! Or you take the next tribe’s pigs in thrilling raids; you grill yams; you trade for televisions and hunt white-plumed birds. Everyone you know agrees: This is the life. Perhaps you burn captives. You set fire to drunks. Yours is the human struggle, or the elite one, to achieve … whatever your own culture tells you: to publish the paper that proves the point; to progress in the firm and gain high title and salary, stock options, benefits; to get the loan to store the beans till their price rises; to elude capture; to feed your children or educate them to a feather edge; to count coup or perfect your calligraphy; to eat the king’s deer or catch the poacher; to spear the seal, intimidate the enemy, and be a big man or beloved woman and die respected for the pigs or the title or the shoes. Not a funeral. Forget funeral. A big birthday party. Since everyone around you agrees ever since there were people on earth that land is value, or labor is value, or learning is value, or title, degree, necklaces, murex shells, the ownership of slaves. Everyone knows bees sting and ghosts haunt and giving your robes away humiliates your rivals. That the enemies are barbarians. That wise men swim through the rock of the earth; that houses breed filth, airstrips attract airplanes, tornadoes punish, ancestors watch, and you can buy a shorter stay in purgatory. The black rock is holy, or the scroll; or the pangolin is holy, the quetzal is holy, this tree, water, rock, stone, cow, cross, or mountain—and it’s all true. The Red Sox. Or nothing at all is holy, as everyone intelligent knows.' , \
                    'Once upon a time I met a stranger and in my mind we lived an entire life together. Not just one life, but many lives. Not just once, but many times. This particular stranger was a lawyer in Portland. I’d come to his office because someone told me there was going to be a protest. There was no protest. I ended up sitting in a corporate conference room with this lawyer in his sage-green fleece vest. We talked about the pipeline that was getting laid near tribal lands in Standing Rock. Beneath his justifications, I could sense a white man’s deep sorrow for the crimes of his country. That sorrow was the part of him I imagined building a life with. My conversation with this lawyer lasted only an hour. But by the time I left, I’d already begun to spin an elaborate fantasy about how our single conversation might change his life. It would force a reckoning with his conscience. He’d remember why he went into law in the first place. He would look up my website! The next time he came to New York, we would have a tryst at a swank Midtown hotel. Maybe the one with the burger joint behind the red velvet curtain. He’d leave his marriage. (The internet told me he was married. But no matter! I was married, too.) The rapid current of my daydream flowed easily around our spouses like two boulders in a river. We would move into a Craftsman bungalow painted the same shade of sage green as his fleece vest. He would save the world by fighting environmental injustice. I would save the world by writing literary essays. We’d watch prestige television once a week. On all the other nights, we wouldn’t need TV; we’d have too much to talk about.  These daydreams occupied the better portion of my flight back east. Back home, they filled all the crevices of my domestic labors. They followed me for weeks, as I walked along the damp sidewalks of Brooklyn, past brownstones whose lit windows exposed perfect lives in which no one needed to dream about living anywhere else. Which was my ultimate daydream: imagining a life in which I no longer daydreamed at all. I’ve spent my whole life daydreaming. It embarrasses me to think of tallying the hours. It feels like ingratitude. It feels like infidelity. It’s often been about infidelity. I’ve daydreamed while walking, while running, while drinking, while smoking—sitting in the Boston cold, seventeen years old, daydreams sprouting like so many weeds from the cracked sidewalk of a broken heart. I’ve daydreamed on every form of transport—something about commuting feels conducive to daydreaming, the pockets of time in between our commitments, and the fact of the body in motion, neither here nor there, available for an elsewhere. I’ve daydreamed to music and in silence, in solitude and in company. It’s hardly exceptional. Studies have found that daydreaming accounts for between a quarter and a half of our waking hours, that we do it every few minutes, during nearly every activity except sex. Eric Klinger, a psychologist at the University of Minnesota, claims that daydreams make up about half the average person’s thoughts. My first daydreams were crushes. In one fantasy, the blond baseball player from my physics class walked up to the front of the room and announced—against a backdrop of velocity and momentum equations—that he wanted to go to prom with me. “She might seem quiet,” he would tell the class. “But once you get to know her, she really has so much to say.” Sometimes I’d revise midstream: He wouldn’t say, She really has so much to say. That was something my middle-aged mom would say. He’d say, She’s pretty awesome.', \
                    'Like a lot of men, in pursuit of novelty and amusement during these months of isolation, I grew a mustache. The reviews were predictably mixed and predictably predictable. “Porny”? Yes. “Creepy”? Obviously. “ ’70s”? True (the 18- and 1970s). On some video calls, I heard “rugged” and “extra gay.” Someone I love called me “zaddy.” Children were harsh. My 11-year-old nephew told his Minecraft friends that his uncle has this … mustache; the midgame disgust was audible through his headset. In August, I spent two weeks with my niece, who’s 7. She would rise each morning dismayed anew to be spending another day looking at the hair on my face. Once, she climbed on my back and began combing the mustache with her fingers, whispering in the warmest tones of endearment, “Uncle Wesley, when are you going to shave this thing off?” It hasn’t been all bad. Halfway through a quick stop-and-chat outside a friend’s house in July, he and I removed our masks and exploded at the sight of each other. No way: mustache! I spent video meetings searching amid the boxes for other mustaches, to admire the way they enhance eyes and redefine faces with a force of irreversible handsomeness, the way Burt Reynolds never made the same kind of sense without his. The mustache aged me. (People didn’t mind letting me know that, either.) But so what? It pulled me past “mature” to a particular kind of “distinguished.” It looks fetching, for instance, with suits I currently have no logical reason to wear. One afternoon, on a group call to celebrate a friend’s good news, somebody said what I didn’t know I needed to hear. More reviews were pouring in (thumbs down, mostly), but I was already committed at that point. I just didn’t know to what. That’s when my friend chimed in: “You look like a lawyer for the N.A.A.C.P. Legal Defense Fund!” What I remember was laughter. But where someone might have sensed shade being thrown, I experienced the opposite. A light had been shone. It was said as a winking correction and an earnest clarification. Y’all, this is what it is. The call moved on, but I didn’t. That is what it is: one of the sweetest, truest things anybody had said about me in a long time. My friend had identified a mighty American tradition and placed my face within it. Any time 20th-century Black people found themselves entangled in racialized peril, anytime the roots of racism pushed up some new, hideous weed, a thoughtful-looking, solemn-seeming, crisply attired gentleman would be photographed entering a courthouse or seated somewhere (a library, a living room) alongside the wronged and imperiled. He was probably a lawyer, and he was likely to have been mustached. Image', \
                    'Writing this in San Francisco, having just come back from San Diego and a heroin Christmas at my mother’s. Not that I used any: there was definitely no blowing, horning, tasting, fixing, goofing, getting loaded or laying out. I’ve always been afraid of serious drugs, knowing my grip on ‘things being OK’ was pretty tenuous already. Back in high school in the early 1970s, when everyone else was dropping acid, I refrained – mainly out of fear that I would be the inevitable freak with no friends who would end up curled up for life in a psychotic ball, or else spattered in ribbony pieces, having flung myself through a plate-glass window. I also wanted to get perfect grades. No: the major dissipations this holiday were candy and coffee and buying things online with just one click. Before I left, Blakey had given me some chocolate cigarettes, and at night, lying on my back under the covers with the laptop on my stomach – my mother had put me in the little upstairs room that used to be Jeff’s – I would reach over and unroll one in a smarmy, bourgeois, sugar-dazed languor. It was a heroin Christmas because I was reading the greatest book I’ve ever read: the jazz musician Art Pepper’s 1979 autobiography Straight Life: The Story of Art Pepper. It knocked my former top pick – Clarissa – right out of first place. As Art himself might say, my joint is getting big just thinking about it. I realise there may be a few lost souls who’ve never heard of him. Forget the overrated (and vapid-looking) Chet Baker. Art Pepper (1925-82) was an authentic American genius. One of the supreme alto saxophone players of all time – Charlie Parker included. A deliriously handsome lover boy in the glory days of his youth. A lifelong dope addict of truly satanic fuck-it-all grandeur. A natural writer of brazen, comic, commanding virtuosity. A proud long-term denizen of the California prison system. And now, no doubt, a tranquil, if desiccated corpse. As his third and last wife, Laurie, notes in the epilogue to Straight Life: ‘Art ... was afraid to be buried in the ground; he was afraid of the worms. But he was terrified of fire. So I had him interred in a crypt at the Hollywood Cemetery, like Rudolph Valentino. He would have enjoyed the location, the company, and that creepy word, crypt.’ If my mother – now 77, curious and freakishly adept at Internet navigation – ever Googles me and sees what I’m writing now, I doubt if she’ll be pleased. Some of my liking, I confess, arises from sheer Southern California white trash fellow feeling. Pepper was born near Los Angeles and spent most of his rackety life (as I have) on the West Coast. His father was a shipyard worker and nasty alcoholic; his mother, a dim-witted teenage bride, didn’t want him and late in her pregnancy tried to abort him by leaping off a table. For what it’s worth, my cross-eyed stepsister Lynne did something similar when she got pregnant at 16 in San Diego in 1967: made my stepsister Laura (who was ten at the time) jump off a sofa onto her stomach to make it ‘pop’. The gambit failed to produce the desired effect. Lynne had the baby in a Catholic place for unwed mothers and left it with an adoption agency. She later married Bill, a telephone installer from San Bernardino with a mammoth, Nietzsche-style moustache, and became a compulsive gambler and grocery-coupon clipper before dying of drink at 46 in 1996. Unlike Art, however, I never mastered a musical instrument. (Plinking guitar accompaniments to ‘Puff the Magic Dragon’ and ‘Love Is Blue’ – grimly laid down as puberty loomed – don’t count.) Pepper was a child prodigy. Though neglected and unloved (his parents split up and basically dumped him) he got hold of a clarinet and taught himself to play. By the age of 14 he was sitting in on clarinet and sax in jazz clubs all around LA. After a short stint in the Army – Pepper was stationed in London after D-Day – he joined the Stan Kenton Orchestra and travelled around the country as Kenton’s lead alto. You can hear one of his very first recorded solos – brief, free, characteristically ductile – three breaks in after the famous scat-singing vocal by June Christy on Kenton’s thunderous account of ‘How High the Moon’ on the recent Proper Box compilation, Bebop Spoken Here. After his first small-group recordings in the early 1950s, discerning jazz fans recognised Pepper as a post-bop player of unusual beauty, subtlety and warmth. The fact that he was white, like several other major West Coast jazz musicians, was not generally held against him. Astonishing to discover, especially given how few people outside music know much about him now, that he came second after Charlie Parker in a 1951 Down Beat readers’ poll for Best Alto Sax Player Ever. Even the most partisan Bird-fanciers acknowledged that Pepper’s tone was the most ravishing ever heard on alto. Parker received 957 votes and Art almost tagged up with 945.', \
                    'Recently I received an e-mail that wasn’t meant for me, but was about me. I’d been cc’d by accident. This is one of the darker hazards of electronic communication, Reason No. 697 Why the Internet Is Bad — the dreadful consequence of hitting “reply all” instead of “reply” or “forward.” The context is that I had rented a herd of goats for reasons that aren’t relevant here and had sent out a mass e-mail with photographs of the goats attached to illustrate that a) I had goats, and b) it was good. Most of the responses I received expressed appropriate admiration and envy of my goats, but the message in question was intended not as a response to me but as an aside to some of the recipient’s co-workers, sighing over the kinds of expenditures on which I was frittering away my uncomfortable income. The word “oof” was used. I’ve often thought that the single most devastating cyberattack a diabolical and anarchic mind could design would not be on the military or financial sector but simply to simultaneously make every e-mail and text ever sent universally public. It would be like suddenly subtracting the strong nuclear force from the universe; the fabric of society would instantly evaporate, every marriage, friendship and business partnership dissolved. Civilization, which is held together by a fragile web of tactful phrasing, polite omissions and white lies, would collapse in an apocalypse of bitter recriminations and weeping, breakups and fistfights, divorces and bankruptcies, scandals and resignations, blood feuds, litigation, wholesale slaughter in the streets and lingering ill will. This particular e-mail was, in itself, no big deal. Tone is notoriously easy to misinterpret over e-mail, and my friend’s message could have easily been read as affectionate head shaking rather than a contemptuous eye roll. It’s frankly hard to parse the word “oof” in this context. And let’s be honest — I am terrible with money, but I’ve always liked to think of this as an endearing foible. What was surprisingly wounding wasn’t that the e-mail was insulting but simply that it was unsympathetic. Hearing other people’s uncensored opinions of you is an unpleasant reminder that you’re just another person in the world, and everyone else does not always view you in the forgiving light that you hope they do, making all allowances, always on your side. There’s something existentially alarming about finding out how little room we occupy, and how little allegiance we command, in other people’s heads. This experience is not a novelty of the information age; it’s always been available to us by the accident of overhearing a conversation at the wrong moment. I’ve written essays about friends that I felt were generous and empathetic, which they experienced as devastating. I’ve also been written about, in ways I could find no fault with but that were nonetheless excruciating for me to read. It is simply not pleasant to be objectively observed — it’s like seeing a candid photo of yourself online, not smiling or posing, but simply looking the way you apparently always do, oblivious and mush-faced with your mouth open. It’s proof that we are visible to others, that we are seen, in all our naked silliness and stupidity. Needless to say, this makes us embarrassed and angry and damn our betrayers as vicious two-faced hypocrites. Which, in fact, we all are. We all make fun of one another behind one another’s backs, even the people we love. Of course we do — they’re ridiculous. Anyone worth knowing is inevitably also going to be exasperating: making the same obvious mistakes over and over, dating imbeciles, endlessly relapsing into their dumb addictions and self-defeating habits, blind to their own hilarious flaws and blatant contradictions and fiercely devoted to whatever keeps them miserable. (And those few people about whom there is nothing ridiculous are by far the most preposterous of all.)' , \
                    'My mother had raised me vegetarian, and though I harbored no real desire to eat meat, sometimes, in summer, I would take a hunk of watermelon to a remote corner of our yard and pretend it was a fresh carcass. On all fours, I would bury my face in the sweet red fruit-meat and tear away mouthfuls. Sometimes, I’d rip handfuls out and cram them in my mouth, which wasn’t much like the way any animal I knew of ate. I was less playing a particular kind of animal than enacting a form of wildness that I recognized in myself. I watched Wild America, a PBS show on which conservationist Marty Stouffer revealed the wildness of the animal world. Alone in the woods behind our house I had beaten my chest, acted out my own invented stories without a thought to how another’s gaze might see me. I sympathized with the jittery business of squirrels and fanatical obsessions of our golden retriever. I was confounded by silverware—why it should exist when we had such perfect instruments at the ends of our arms. Walt Whitman claimed our distinction from animals to be that “they do not sweat and whine about their condition” and “not one is dissatisfied, not one is demented with the mania of owning things.” However often Stouffer imposed human narratives on the animals depicted (very often), it was still always clear that survival was the priority that assigned value to everything in the animal world. If the wild marten was overcome by her own feelings, she didn’t let it stop her from procuring dinner for her babies. I might have had to close my eyes during the part of the nature documentary in which the pack of hyenas felled an antelope, but they had no qualms about tearing warm mouthfuls from her while she still kicked with frantic life. I learned in elementary school that we were animals, but unlike other animals we did not seem driven by the instinct for physical survival. We were so far up the food chain that it was no longer even visible to us. We were beyond survival, in a dark and lofty realm wherein our obsolete instincts had been perverted into atrocities like capitalism and bikini waxing. I might not have been able to name this, but I recognized it. Sometimes, when I momentarily detached from the narrative of human life that we all took for granted—the one that presumes that money, cars, shopping malls, pollution, and industry are not a demented and catastrophic misuse of our resources—and glimpsed it from an evolutionary angle, it seemed so bizarre as to be unlikely. Was this real life or some strange dystopian movie, a dream from which we would soon wake to resume our sensible animal lives—in which “nature” was not a television show category or an experience to cultivate a preference for consuming but the only thing, the everything.', \
                    'When I was twenty years old, I became a kind of apprentice to a man named Andrew Lytle, whom pretty much no one apart from his negligibly less ancient sister, Polly, had addressed except as Mister Lytle in at least a decade. She called him Brother. Or Brutha—I don’t suppose either of them had ever voiced a terminal r. His two grown daughters did call him Daddy. Certainly I never felt even the most obscure impulse to call him Andrew, or “old man,” or any other familiarism, though he frequently gave me to know it would be all right if I were to call him mon vieux. He, for his part, called me boy, and beloved, and once, in a letter, “Breath of My Nostrils.” He was about to turn ninety-two when I moved into his basement, and he had not yet quite reached ninety-three when they buried him the next winter, in a coffin I had helped to make—a cedar coffin, because it would smell good, he said. I wasn’t that helpful. I sat up a couple of nights in a freezing, starkly lit workshop rubbing beeswax into the boards. The other, older men—we were four altogether—absorbedly sawed and planed. They chiseled dovetail joints. My experience in woodworking hadn’t gone past feeding planks through a band saw for shop class, and there’d be no time to redo anything I might botch, so I followed instructions and with rags cut from an undershirt worked coats of wax into the cedar until its ashen whorls glowed purple, as if with remembered life. The man overseeing this vigil was a luthier named Roehm whose house stood back in the woods on the edge of the plateau. He was about six and a half feet tall with floppy bangs and a deep, grizzled mustache. He wore huge glasses. I believe I have never seen a person more tense than Roehm was during those few days. The cedar was “green”—it hadn’t been properly cured. He groaned that it wouldn’t behave. On some level he must have resented the haste. Lytle had lain dying for weeks; he endured a series of disorienting pin strokes. By the end they were giving him less water than morphine. He kept saying, “Time to go home,” which at first meant he wanted us to take him back to his house, his real house, that he was tired of the terrible simulacrum we’d smuggled him to, in his delirium. Later, as those fevers drew together into what seemed an unbearable clarity, like a blue flame behind the eyes, the phrase came to mean what one would assume. He had a deathbed, in other words. He didn’t go suddenly. Yet although his family and friends had known for years about his wish to lie in cedar, which required that a coffin be custom made, no one had so much as played with the question of who in those mountains could do such a thing or how much time the job would take. I don’t hold it against them—against us—the avoidance of duty, owing as it did to fundamental incredulity. Lytle’s whole existence had for so long been essentially posthumous, he’d never risk seeming so ridiculous as to go actually dying now. My grandfather had told me once that when he’d been at Sewanee, in the thirties, people had looked at Lytle as something of an old man, a full sixty years before I met him. And he nursed this impression, with his talk of coming “to live in the sense of eternity,” and of the world he grew up in—Middle Tennessee at the crack of the twentieth century—having more in common with Europe in the Middle Ages than with the South he lived to see. All of his peers and enemies were dead. A middle daughter he had buried long before. His only wife had been dead for thirty-four years, and now Mister Lytle was dead, and we had no cedar coffin. ', \
                    'For a year I worried that I might die having had sexual intercourse with only one person. Like many college students, I thought intercourse was the greatest thing in life. And it just about killed me to hear of everyone’s rambunctious affairs while I was in the hospital. When I was 21, in the mid-’90s, my immune system filled my blood with poison. It was trying to destroy my nervous system  a misperception that caused me a lot of trouble. When a medical student came by my hospital room after his rounds were over, with a book or a mix tape he’d recorded, I thought about inviting him to sleep with me if they told me I would die there in the hospital. But I went back to college in spring still sick, with a tube still implanted in my chest. It looked like something I should pull out  a white dart, a poison arrow  but I couldn’t pull it out. I needed the big tube so my blood could be removed and cleaned and put back with the cells intact. I’d had it done with big needles, wide bores, in the crooks of my elbows, but my arm veins had long since blown. Then I had three catheters in my subclavian vein. The scar tissue from those still feels like hard cranberries buried under my collarbone. The permanent line was a 20-inch tube that never clogged as long as blood thinner was shot into it every two days. From one direction it went into my right breast, under the collarbone and up, just under the skin, into my jugular, and then into my subclavian vein and toward my heart. On the outside it hung like two white drinking straws, six inches long, with one red clamp and one blue one, like a piece of jewelry. I gave away my scoop-necked sweaters. I used big clear adhesive patches to cover the entry site when I showered. The catheter never got torn out in my sleep because I learned how to roll over slowly, even in deep sleep, while cradling the catheter in my left hand. Every month or so I went back to the hospital to have my plasma replaced or to have a bag of medicine infused. My blood was removed and cleaned and put back more than 50 times. After that, my hematologist tried another treatment: massive gamma-globulin infusions. The second infusion kept me going for three months, and it was decided I wouldn’t have to have my plasma replaced again. My neurologist said I’d turned a corner, so after 11 1/2 months, my central line was pulled. I believed, though, that I would stop secreting antibodies only after I had sexual intercourse. And though I looked worse than I ever had in my life, thanks to the steroids  I was fat and swollen, covered in acne, and had a gruesomely round face  I thought my legendarily promiscuous musician friend might still be interested. Editors’ Picks. 18,000 Miles Later, an American Woman Has Cycled the World. The Origin Story of Astro Bot’s Likable Little Guy.Overlooked No More: Gwendolyn B. Bennett, Harlem Renaissance Star Plagued by Misfortune. So I called him and invited him to have a drink with me that night. We had our drink and walked back to our dorm and sat down in the courtyard, just talking. It was June, two days before commencement. He was graduating, and I was graduating, too, sort of, but the envelope I was getting wouldn’t have a diploma in it. I had another semester of course work to complete. Only the seniors were left at school, and most of them were in the courtyard. I felt exposed. Finally, getting up from the bench we’d been sitting on, my friend said, “Your place?” And we went to my dorm room, which was a single suite I had all to myself, with my own bathroom, because my neurologist had written a note to the university. We sat on my futon, drinking out of a plastic bottle of vodka. Eventually he said, “Do you have any other rooms in this place?” and walked me to the bedroom, and lay me on my bed, and had intercourse with me. Then he asked me about the scabs on my chest from where the line had just been pulled out and listened to the things I told him, and held me very tightly. Two mornings later, when we were in the courtyard again, seated in rows for graduation, he was wearing a buttoned shirt and sweating, because his neck was covered with bite marks. Years passed. He and I wrote almost every day. I lived in one city and he lived in another. He told me some of his secrets, and I told him some of mine. Our letters were intimate, but I didn’t get around to explaining to him that I recovered from my disease only because he had selflessly had intercourse with an ugly version of a girl he once had a crush on.', \
                    'What, to begin with, was Baldwin’s relation to his father, and what was his legacy from his father? What did he learn in his year living on his own in New Jersey, and what was the “dread, chronic disease” that he first contracted there? What does Baldwin learn about his father from the funeral? What does he mean when he writes, “It was the Lord who knew of the impossibility every parent in that room faced: how to prepare the child for the day when the child would be despised and how to create in the child—by what means?—a stronger antidote to this poison than one had found for oneself”? What enables him to say, and why does he say, “blackness and whiteness did not matter; to believe that they did was to acquiesce in one’s own destruction”? What is his final judgment about hatred, and about the proper stance toward injustice? On the 29th of July, in 1943, my father died. On the same day, a few hours later, his last child was born. Over a month before this, while all our energies were concentrated in waiting for these events, there had been, in Detroit, one of the bloodiest race riots of the century. A few hours after my father’s funeral, while he lay in state in the undertaker’s chapel, a race riot broke out in Harlem. On the morning of the 3rd of August, we drove my father to the graveyard through a wilderness of smashed plate glass. The day of my father’s funeral had also been my nineteenth birthday. As we drove him to the graveyard, the spoils of injustice, anarchy, discontent, and hatred were all around us. It seemed to me that God himself had devised, to mark my father’s end, the most sustained and brutally dissonant of codas. And it seemed to me, too, that theviolence which rose all about us as my father left the world had been devised as a corrective for the pride of his eldest son. I had declined to believe in that apocalypse which had been central to my father’s vision; very well, life seemed to be saying, here is something that will certainly pass for an apocalypse until the real thing comes along. I had inclined to be contemptuous of my father for the conditions of his life, for the conditions of our lives. When his life had ended I began to wonder about that life and also, in a new way, to be apprehensive about my own. I had not known my father very well. We had got on badly, partly because we shared, in our different fashions, the vice of stubborn pride. When he was dead I realized that I had hardly ever spoken to him. When he had been dead a long time I began to wish I had. It seems to be typical of life in America, where opportunities, real and fancied, are thicker than anywhere else on the globe, that the second generation has no time to talk to the first. No one, including my father, seems to have known exactly how old he was, but his mother had been born during slavery. He was of the first generation of free men. He, along with thousands of other Negroes, came north after 1919 and I was part of that generation which had never seen the landscape of what Negroes sometimes call the Old Country. ', \
                    'Does there come a day in every man’s life when he looks around and says to himself, “I’ve got to weed out some of these owls”? I can’t be alone in this, can I? And, of course, you don’t want to hurt anyone’s feelings. Therefore you keep the crocheted owl given to you by your second-youngest sister and accidentally on purpose drop the mug that reads “Owl Love You Always” and was sent by someone who clearly never knew you to begin with. I mean, mugs with words on them! Owl cocktail napkins stay, because everyone needs napkins. Ditto the owl candle. Owl trivet: take to the charity shop along with the spool-size Japanese owl that blinks his eyes and softly hoots when you plug him into your computer. Just when you think you’re making progress, you remember the owl tobacco tin, and the owl tea cozy. Then there are the plates, the coasters, the Christmas ornaments. This is what happens when you tell people you like something. For my sister Amy, that thing was rabbits. When she was in her late thirties, she got one as a pet, and before it had chewed through its first phone cord she’d been given rabbit slippers, rabbit cushions, bowls, refrigerator magnets, you name it. “Really,” she kept insisting, “the live one is enough.” But nothing could stem the tide of crap. Amy’s invasion started with a live rabbit, while Hugh’s and mine began, in the late nineteen-nineties, with decorative art. We were living in New York then, and he had his own painting business. One of his clients had bought a new apartment, and on the high, domed ceiling of her entryway she wanted a skyful of birds. Hugh began with warblers and meadowlarks. He sketched some cardinals and blue tits for color and was just wondering if it wasn’t too busy when she asked if he could add some owls. It made no sense nature-wise—owls and songbirds work different shifts, and even if they didn’t they would still never be friends. No matter, though. This was her ceiling, and if she wanted turkey vultures—or, as was later decided, bats—that’s what she would get. All Hugh needed was a reference, so he went to the Museum of Natural History and returned with “Understanding Owls.” The book came into our lives almost fifteen years ago and I’ve yet to go more than a month without mentioning it. “You know,” I’ll say, “there’s something about nocturnal birds of prey that I just don’t get. If only there was somewhere I could turn for answers.” “I wish I could help you,” Hugh will say, adding, a second or two later, “Hold on a minute . . . what about . . . ‘Understanding Owls’?” We’ve performed this little routine more times than I can count, but back then, when the book was still fresh-smelling and its pages had not yet yellowed, I decided that because Hugh actually did get a kick out of owls, I would try to find him a stuffed one. My search turned up plenty of ravens. I found pheasants and ducks, and foot-tall baby ostriches. I found a freeze-dried turkey’s head attached to its own foot, but owls, no luck. That’s when I learned that it’s illegal to own them in the United States. Even if one dies naturally, of a stroke or old age. If it chokes on a mouse or gets kicked by a horse. Should one fly against your house, break its neck, and land like magic on your front stoop, you’re still not allowed to stuff it or even to store its body in your freezer. Technically, you’re not even allowed to keep one of its feathers—that’s how protected they are. I learned this at a now defunct taxidermy shop in midtown Manhattan. “But, if you’re really interested,” the clerk I spoke to said, “I’ve got a little something you might want to see.” He stepped into the back room and returned with what I could only identify as a creature. “What we’ve done,” he boasted, “is stretch a chicken over an owl form.”'
                ]


# AI + paraphraser
string_ai_para = [
                    'The distinction between information produced by machines and humans has become more hazy due to artificial intelligence\'s (AI) explosive growth. While AI has transformed many industries, separating AI-generated material from human-generated language has proven to be a major difficulty. The intricacy of human language, the sophistication of AI language models, and the subjective nature of evaluation are some of the interrelated reasons contributing to this challenge.A major contributing factor to the challenge of identifying text produced by AI is the growing complexity of AI language models. Large datasets of text written by humans are used to train these models, such GPT-3 and LaMDA, so they can pick up on the nuances, patterns, and styles of human language. Because of this, they are able to generate writing that is strikingly similar to information created by humans, making it difficult to tell which text is artificial. In certain circumstances, these models may even produce cohesive and creative prose, frequently outperforming human abilities.In addition, the intricacy of human language itself adds to the challenge of identification. Ambiguities, subtleties, and meanings that vary depending on the context are common in human language. It is frequently distinguished by idioms, figurative language, and cultural allusions that might be difficult for AI models to fully understand and reproduce. Even while AI models have come a long way in comprehending and producing language that is similar to that of humans, they can still find it difficult to convey the nuances and quirks of human communication.A further obstacle to differentiating AI-generated material from text produced by humans is the subjective nature of judgement. Individual interpretations, cultural preferences, and personal biases frequently impact the evaluation of text quality. What one individual may deem to be excellent writing may not be the same for another. Because of this, establishing objective standards for differentiating between material written by AI and human authors might be difficult.In summary, the intricacy of human language, the sophistication of AI language models, and the subjective nature of assessment all contribute to the challenge of identifying text produced by artificial intelligence. Even though AI has come a long way in producing text that resembles that of a human, it is far from flawless. It\'s possible that the difficulty of identifying human-generated writing from AI-generated stuff may increase as AI develops further.',
                    'For thousands of years, philosophers, physicists, and mystics have been enthralled with consciousness—that mysterious, subjective domain of experience. It\'s still one of the biggest mysteries of the cosmos. How can the rich tapestry of ideas, emotions, and sensations that we refer to as consciousness arise from a group of neurones firing electrical signals?We are still in the early stages of scientific study of consciousness. Research on the neurobiology of consciousness has revealed the involvement of specific brain regions in functions including self-awareness, memory, and attention. The specific mechanisms driving these processes are still unknown, though. While some theories contend that consciousness is an innate feature of the cosmos, others contend that it develops from intricate patterns of brain activity.Consciousness has also been studied in philosophical and spiritual traditions outside of science.Numerous philosophers have maintained that awareness is a special quality of human life that sets it apart from the physical universe. Certain spiritual and religious traditions hold that awareness exists outside of the body and is linked to a higher spiritual plane.Deep philosophical enquiries concerning the nature of reality, the self, and the purpose of existence are brought up by the question of consciousness. Is awareness something more, or just a byproduct of the brain? Is consciousness attainable for machines? What connection exists between free will and consciousness?We might discover that consciousness is not a single phenomena but rather the result of a complex interaction between biological, psychological, and possibly even metaphysical aspects as we continue to investigate its mysteries. The quest to understand consciousness is a journey that takes us to the very heart of what it means to be human.',
                    'Recent years have seen notable developments in particle physics, the study of matter\'s basic components and interactions. These discoveries have been made possible in large part by the Large Hadron Collider (LHC), the strongest particle accelerator yet constructed. The confirmation of the presence of the Higgs boson, a particle predicted by the Standard Model of particle physics, was one of the most important discoveries made possible by the LHC. Understanding how other particles acquire mass depends on the existence of the Higgs boson. Its finding was a significant victory for the Standard Model, offering solid proof of its viability. The LHC has probed particle physics in areas other than the Higgs boson. In order to expand the Standard Model and provide an explanation for phenomena like dark matter and dark energy, scientists have been looking for evidence of new particles and forces. Even if conclusive findings have not yet been made in this field, the LHC keeps pushing the boundaries of our knowledge. Particle physics has a bright future ahead of it, full with fascinating possibilities. With planned modifications to boost its energy and luminosity, the LHC is likely to remain operational for many years to come. These improvements may result in fresh findings and a better comprehension of nature\'s basic principles. Other particle physics experiments are also contributing significantly, in addition to the LHC. The properties of neutrinos, elusive particles that can nearly completely evade detection in matter, are being investigated through neutrino experiments. The bulk of matter in the cosmos is made up of dark matter particles, which have not yet been directly detected. Dark matter experiments are looking for evidence of these particles. We should anticipate more developments in our comprehension of the underlying components of the cosmos as particle physics continues to advance. The advancements in this discipline could fundamentally alter our understanding of the universe and influence technological developments in the future.', 
                    'Education systems in different countries reflect their particular societal values, priorities, and historical backgrounds. For instance, the systems in the USA, Finland, and India provide radically diverse approaches and structures, each with advantages and disadvantages of their own. Although it would be easy to rank these systems and declare one to be the "best," these comparisons sometimes leave out crucial contextual and cultural elements. America: Variety and Adaptability. The U.S. education system is characterized by its diversity, flexibility, and emphasis on individualism. It offers a variety of learning environments by letting parents and students select from public, private, charter, and homeschooling alternatives. The United States of America is a leader in higher education, with several of its universities being among the best in the world and encouraging research and innovation. On the other hand, there are significant differences in funding, quality, and access to resources at the K–12 level, particularly between wealthy and low-income districts. A significant component of assessment is standardised testing, which has drawn criticism for constricting curriculum and emphasising test performance above comprehensive learning. Finland: Welfare and Equality. The focus Finland places on equity, student well-being, and teacher trust is well known. Finland\'s schools offer a laid-back learning atmosphere that prioritises creativity, teamwork, and critical thinking above standardised testing. Teachers are very autonomous in creating their own lessons because they are well-respected experts. In Finland, formal education begins later in life and is centred on stress reduction and lifetime learning. Finland\'s education system has come under fire for its sluggishness in responding to the changing demands of a more digital and globalised world, despite the country often placing highly in international education examinations such as PISA (Programme for International Student Assessment). India: Dimensions and Goals. India\'s large population and wide range of socioeconomic backgrounds present special opportunities and difficulties for the country\'s educational system. The Right to Education Act, which requires free and compulsory education for children aged 6 to 14, is one of the policies that has helped the nation make tremendous progress in expanding access to education. High-stakes tests and rote learning are heavily emphasised in the Indian educational system, particularly in secondary education. Nonetheless, there are notable differences between rural and urban locations, and developing nations sometimes have less access to high-quality education. In spite of these obstacles, India generates a lot of STEM graduates who support innovation and the global tech sector. Why It Is Wrong to Call Any One System the Best. Any claim that one of these systems is the "best" ignores the fact that the educational models of other nations are influenced by distinct social, cultural, and economic circumstances. India needs to address issues related to its large population and resource imbalances, Finland emphasises equity and teacher autonomy, while the United States encourages choice and flexibility. Every system has both obstacles that are firmly anchored in historical and socioeconomic reasons and strengths that correspond with the needs of the society in which it operates. Moreover, what works in one country may not be effective or relevant in another due to differences in culture, government structure, and societal goals. In conclusion, education is not one-size-fits-all. Instead of ranking systems, it’s more productive to analyze each within its own context and explore ways to learn from their successes and failures.',
                    'Art has changed dramatically over the past 500 years as a result of shifting philosophical perspectives, technological breakthroughs, and changing societal values. During the 16th-century Renaissance, ideal beauty, humanism, and scientific accuracy were given priority by artists such as Michelangelo and Leonardo da Vinci. With artists like Caravaggio and Goya expressing strong human experiences, art portrayed grandeur, emotion, and force as it transitioned into the Baroque and Romantic periods. With the rise of Impressionism and later Modernism in the late 19th century, artists like as Monet and Picasso redefined the parameters of art by experimenting, emphasising personal expression, and breaking away from conventional forms of realism. A pluralistic approach, embracing a variety of genres like Surrealism, Abstract Expressionism, and Minimalism, where artists like Dalí and Pollock further pushed the boundaries of form and concept, was introduced in the 20th century. With the advent of digital technology and globalisation, contemporary art has challenged traditional ideas of authorship and originality by experimenting with a wide range of media, including performance art and digital installations. Thanks to social media and international platforms, artists may now reach a wide range of people, making art more democratic and accessible than ever. The distinction between fine art and popular culture is becoming increasingly hazy, since what was previously limited to affluent patrons and institutions has evolved into a kind of mass cultural expression. Art is no longer bound by rigid definitions; instead, it thrives as an ever-evolving reflection of society’s complexities, ideas, and technological advances, making it impossible to declare a singular approach or movement as definitive.',
                    'The intricate interactions between major world powers, economic interdependence, technological breakthroughs, and growing nationalism characterise the geopolitical landscape of the twenty-first century. The globe now is multipolar, with the United States, China, and Russia each playing prominent but different roles—a stark contrast to the bipolar world of the Cold War era. Although China is quickly becoming a major economic force in the world, a leader in technology, and a military force in Asia, particularly in the South China Sea, the United States is still the dominant military and economic force in the region. Despite having a smaller economy, Russia nevertheless has a lot of power thanks to its energy exports, military actions, and cyber capabilities. This is especially true given its participation in Ukraine and the larger post-Soviet region. In the meantime, the European Union remains a significant diplomatic and economic force, particularly when it comes to tackling issues related to human rights and climate change, despite internal obstacles like Brexit and divergent national interests. The 21st century has also seen an increase in the significance of developing markets as major actors in international trade and diplomacy, especially South Africa, Brazil, and India. Nonetheless, a backlash against globalisation is reflected in the growth of populist and nationalist parties throughout the world, particularly in the United States, Europe, and Latin America. This has resulted in an increase in protectionism, the implementation of anti-immigration laws, and the deterioration of multilateral organisations like the UN and WTO. Geopolitical tensions have increased due to climate change, which has made resource conflicts, migration, and natural disasters worse. Global power conflicts have taken on new dimensions due to technical breakthroughs, namely in the areas of artificial intelligence, cyber warfare, and space exploration. With major powers fighting for dominance in the tech sector, cybersecurity, information warfare, and control over digital infrastructure have become crucial to national security. Thus, the geopolitical environment is defined by a global power struggle that is increasingly occurring in both the physical and digital spheres, as well as by shifting alliances, regional conflicts, and the growth of authoritarianism. In the twenty-first century, global stability is precarious and collaboration and rivalry coexist. To meet these unprecedented challenges, new models of governance and diplomacy are needed. ',
                    'Dedicated oneself solely to using computers There are benefits and drawbacks to gaming, including effects on social, mental, and physical health. Positively, playing games can improve cognitive abilities like hand-eye coordination, strategic thinking, and problem-solving, particularly in fast-paced, intricate games that call for multitasking. A sense of accomplishment, stress reduction, and social contact are also brought about by it. This is especially true of multiplayer games, where players can compete or collaborate with people all over the world, promoting a sense of community and teamwork. Some even use professional eSports or content creation to make gaming their career. Nevertheless, playing video games too much has serious drawbacks. Physically, spending extended amounts of time in front of a computer can cause sedentary behaviour, which raises the risk of obesity, bad posture, eye strain, and injuries from repeated strain. Mentally, playing video games can develop to addiction, which can cause one to overlook obligations in the real world such social interactions, employment, or education. An over-reliance on video games for social contact might erode real social engagement and face-to-face communication abilities. Furthermore, playing video games too much can exacerbate mental health conditions like stress, worry, and depression—especially if it\'s done as a coping method. Therefore, in order to avoid gaming\'s detrimental effects on health and wellbeing, it must be balanced with physical activity, social engagement, and other obligations, even though gaming provides enjoyment and cognitive benefits. ',
                    'The field of artificial intelligence chatbots has been rapidly changing, and three major competitors—Gemini, Claude, and ChatGPT—have emerged, each with their own advantages and skills. With its remarkable natural language interpretation and generation capabilities, ChatGPT, created by OpenAI, first raised the bar for big language models. It also gained a lot of popularity and demonstrated the promise of AI in a variety of applications. Anthropic\'s creation, Claude, came onto the scene emphasising safety and moral issues. He had sophisticated reasoning abilities and a nuanced approach to difficult subjects. With the integration of multimodal capabilities, Google\'s newest AI model, Gemini, seeks to go above previous limits by processing and producing both text and visual material with ease. All three chatbots are excellent at general conversation and task accomplishment, but they differ in a few key ways. ChatGPT is well-known for its adaptability and depth of knowledge, which enable it to handle a variety of jobs, from help with coding to creative writing. Claude distinguishes itself by emphasising safety and conformity to human values, frequently offering more circumspect and morally conscious solutions. Gemini has an advantage in tasks including both textual and visual features thanks to its multimodal approach, which may lead to new applications for AI. Based on current comparisons, it appears that these models perform fairly well, with each model excelling in a distinct domain. ChatGPT is able to stay relevant on a wide range of subjects because of its regular updates and large training dataset. When it comes to difficult thinking exercises and talks of delicate topics, Claude frequently exhibits intelligent and nuanced responses. The multimodal features of Gemini provide special benefits in situations when text generation and visual comprehension are required. But it\'s crucial to remember that the field of artificial intelligence is evolving quickly, and these models are updated and improved on a regular basis. The decision between Gemini, Claude, and ChatGPT may ultimately come down to particular use cases, moral issues, and the ideal ratio of functionality to safety precautions. As these AI chatbots continue to improve, they collectively represent the leading edge of natural language processing technology, each contributing to the advancement of human-AI interaction in its own unique way.',
                    'There are more than a thousand people living in Tristan da Cunha, a series of volcanic islands in the South Atlantic Ocean, making it the most remote place on Earth. This British Overseas Territory holds the distinction of being the world\'s most remote inhabited archipelago, situated roughly 2,088 miles (3,360 kilometres) from South America and 1,500 miles (2,400 kilometres) from the closest continental land mass in South Africa. The entire population of about 250 people lives on the main island, Tristan da Cunha, and they all reside in the settlement of Edinburgh of the Seven Seas. Although it falls short of the 1,000-person requirement, it is noteworthy as the largest community that genuinely reflects extreme remoteness. We might take into consideration Longyearbyen, which has about 2,400 residents and is located on the Norwegian archipelago of Svalbard, as a place that satisfies both the population and remoteness criteria. It is the world\'s northernmost settlement with over a thousand residents, located at 78 degrees north latitude, approximately 650 miles (1,050 km) from the North Pole. Extreme Arctic weather is common in Longyearbyen, with four months of polar nights and an equivalent amount of midnight sun. Originally founded as a coal mining town, the hamlet has developed into a centre for satellite technology, tourism, and Arctic research. The distinctive difficulties of Longyearbyen living include the polar bear threat that must always be considered when leaving the settlement boundaries and carrying a firearm. Infrastructure and daily life are also made more challenging by the permafrost environment. In spite of these difficulties, Longyearbyen continues to be a thriving village with contemporary facilities including an airport, hospital, and academic centre that act as a lifeline to the outside world. Places like Tristan da Cunha and Longyearbyen are extremely isolated, which highlights how adaptable and resilient people can be. These towns\' isolated locations and challenging conditions have shaped their unique cultures and ways of life. Offering distinct viewpoints on sustainability, community, and the interaction between humans and their environment, they stand as a tribute to humanity\'s never-ending spirit of exploration and ability to flourish in even the most remote regions of our globe.',
                    'In the field of quantum many-body physics, machine learning (ML) has become a potent instrument that offers fresh perspectives on solving intricate issues that have long eluded conventional computational techniques. The utilisation of machine learning (ML) techniques, such neural networks and deep learning algorithms, in quantum systems has created novel opportunities for comprehending and replicating complex quantum phenomena involving numerous interacting particles. These machine learning techniques have demonstrated extraordinary success in fields including quantum state tomography, quantum control protocol optimisation, and the discovery of novel quantum phases of matter. Through the utilisation of machine learning models, which are capable of identifying patterns and extracting pertinent information from large datasets, scientists can now investigate quantum systems with a level of efficiency and precision never seen before. Significant progress has been made as a result of the collaboration between ML and quantum physics, including better material property predictions, more effective quantum simulations, and fresh perspectives on quantum phase transitions. There is still debate, nevertheless, regarding the incorporation of machine learning into quantum many-body physics. On the one hand, supporters claim that machine learning (ML) techniques provide a much-needed computational boost to a subject that is frequently constrained by the exponential complexity of quantum systems, with the potential to advance both the development of new quantum technologies and our understanding of fundamental physics. Since machine learning techniques are more adept at handling high-dimensional data than conventional methods, they are especially well-suited for the investigation of entangled quantum states. However, detractors worry whether the black-box character of some ML algorithms might mask the underlying physical principles at work, raising doubts about the interpretability of ML models in physics. There\'s also a discussion concerning the credibility of ML predictions in quantum systems, especially when extrapolating outside the training data. Moreover, some physicists are concerned that an over-reliance on ML could result in a loss of theoretical understanding since researchers might grow overly reliant on data-driven methods at the price of gaining a deeper understanding of fundamental physics. Many academics agree that, in spite of these worries, the advantages of machine learning in quantum many-body physics exceed any possible disadvantages. The secret is to use machine learning (ML) as an additional tool to augment conventional physical procedures and theoretical knowledge, not as a substitute for them. Through a combination of rigorous physical principles, experimental validation, and the benefits of machine learning, scientists can leverage the power of data-driven methodologies without sacrificing the interpretability and physical intuition that are essential to scientific advancement. It will be crucial to create more transparent and physically justified machine learning techniques as the subject develops to make sure that the knowledge acquired using these approaches actually advances our understanding of quantum many-body systems. In the end, the wise use of machine learning in quantum physics is a potentially exciting new field that, with careful exploration, might completely transform our capacity to understand and manipulate the quantum universe.'
                ]

In [8]:
data['text'] = string_gemini + string_chatgpt + string_claude + string_human + string_ai_para
data

,text,generated
0,The rapid advancement of artificial intelligen...,1
1,"Consciousness, that elusive, subjective realm ...",1
2,"Particle physics, the study of the fundamental...",1
3,Education systems in different countries refle...,1
4,"Over the last 500 years, art has undergone a r...",1
5,The geopolitical climate of the 21st century i...,1
6,Spending all one's time on computers gaming pr...,1
7,The landscape of AI chatbots has been rapidly ...,1
8,The remotest inhabited place on Earth with a p...,1
9,Machine learning (ML) has emerged as a powerfu...,1


In [9]:

max_len = 400
test_input_ids, test_attention_masks = tokenize_and_encode( 
    tokenizer, 
    data.text.tolist(), \
    max_length = max_len
) 

test_dataset = TensorDataset(test_input_ids, test_attention_masks) 
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

pred_prob, pred = testing(model, test_loader)

data['pred'] = pred
data['pred_prob'] = pred_prob


data[:]


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2760: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

,text,generated,pred,pred_prob
0,The rapid advancement of artificial intelligen...,1,1,0.942228
1,"Consciousness, that elusive, subjective realm ...",1,1,0.997167
2,"Particle physics, the study of the fundamental...",1,1,0.999472
3,Education systems in different countries refle...,1,1,0.999804
4,"Over the last 500 years, art has undergone a r...",1,1,0.999863
5,The geopolitical climate of the 21st century i...,1,1,0.999961
6,Spending all one's time on computers gaming pr...,1,1,0.999991
7,The landscape of AI chatbots has been rapidly ...,1,1,0.999919
8,The remotest inhabited place on Earth with a p...,1,1,0.510917
9,Machine learning (ML) has emerged as a powerfu...,1,1,0.999898
